## 📁 Data Loading and Library Setup

> This section imports the required libraries and loads the input dataset for further processing.

In [43]:
# Import core libraries for data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [44]:
# Read input CSV file
df = pd.read_csv('dataset.csv')

df.sample(5)

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
4972,4972,BHMMBMMBX01,687,20.000035,78.000003,674,car,high,8,0,02-12-2016,12:58:00
11185,11185,Others-CCCPS119a,2803,26.147541,91.727970,296,car,average,5,1,13-11-2016,10:33:00
10252,10252,Others-CCCPS105a,2009,26.147473,91.728049,1263,bike,average,6,0,06-12-2016,11:02:00
6519,6519,BHMNCPHST01,1200,26.140014,91.731000,634,car,average,4,0,17-12-2016,14:30:00
9982,9982,Others-CCCPS105a,2009,26.147473,91.728049,950,truck,high,7,0,19-11-2016,10:58:00


In [45]:
# Display column names, non-null counts, and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


## 🛠️ Feature Engineering & Transformation

> In this section, we define utility functions to extract and transform meaningful features from the raw dataset for downstream modeling.


In [46]:
# Function to extract parking metadata and create useful mappings
def build_parking_metadata(df):
    """
    Builds:
    - meta_dict: index ➝ {SystemCodeNumber, Capacity, Latitude, Longitude}
    - id_map: SystemCodeNumber ➝ index
    - reverse_id_map: index ➝ SystemCodeNumber
    """
    df = df.copy()

    # Ensure consistent encoding by sorting
    unique_ids = sorted(df['SystemCodeNumber'].unique())
    id_map = {val: idx for idx, val in enumerate(unique_ids)}
    reverse_id_map = {idx: val for val, idx in id_map.items()}

    df['SystemCodeNumberEncoded'] = df['SystemCodeNumber'].map(id_map)

    meta_dict = {}
    for _, row in df.drop_duplicates('SystemCodeNumberEncoded').iterrows():
        idx = row['SystemCodeNumberEncoded']
        meta_dict[idx] = {
            'SystemCodeNumber': row['SystemCodeNumber'],
            'Capacity': row['Capacity'],
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude']
        }

    return meta_dict, id_map, reverse_id_map

In [47]:
# Transform raw parking data: timestamp parsing, feature encoding, and engineering
def transform_parking_data(df, id_map):
    df = df.copy()

    # Combine date and time columns into a single datetime column
    df['datetime'] = pd.to_datetime(
        df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
        format='%d-%m-%Y %H:%M:%S'
    )

    # Round datetime to the nearest 30 minutes and clip to 08:00–16:30
    from datetime import timedelta, time

    def round_to_nearest_30min(dt):
        minute = dt.minute
        if minute < 15:
            rounded_minute = 0
        elif minute < 45:
            rounded_minute = 30
        else:
            dt += timedelta(hours=1)
            rounded_minute = 0
        dt = dt.replace(minute=rounded_minute, second=0, microsecond=0)

        # Clip time range
        min_time = time(8, 0)
        max_time = time(16, 30)
        if dt.time() < min_time:
            dt = dt.replace(hour=8, minute=0)
        elif dt.time() > max_time:
            dt = dt.replace(hour=16, minute=30)
        return dt

    df['TimeStamp'] = df['datetime'].apply(round_to_nearest_30min)
    df = df.sort_values('TimeStamp').reset_index(drop=True)

    # Encode categorical variables
    traffic_map = {'low': 0, 'average': 1, 'high': 2}
    vehicle_map = {'cycle': 0, 'bike': 1, 'car': 2, 'truck': 3}
    df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map(traffic_map)
    df['VehicleType'] = df['VehicleType'].map(vehicle_map)
    df['SystemCodeNumber'] = df['SystemCodeNumber'].map(id_map)

    # Time category (Morning, Noon, Afternoon based on half-hour slots)
    hour = df['TimeStamp'].dt.hour
    minute = df['TimeStamp'].dt.minute
    slot_index = (hour - 8) * 2 + (minute // 30)
    df['TimeCategory'] = slot_index // 6

    # Utilization = Occupancy / Capacity
    df['Utilization'] = df['Occupancy'] / df['Capacity']

    # QueuePressure = log(QueueLength / EmptySpots)
    df['EmptySpots'] = np.maximum(1, df['Capacity'] * (1 - df['Utilization']))
    df['QueuePressure'] = df['QueueLength'] / df['EmptySpots']
    df['QueuePressure'] = np.log1p(df['QueuePressure'])
    df.drop(['EmptySpots'], axis=1, inplace=True)

    # Final feature columns to return
    final_cols = [
        'SystemCodeNumber',
        'VehicleType',
        'TrafficConditionNearby',
        'TimeStamp',
        'IsSpecialDay',
        'TimeCategory',
        'Occupancy',
        'Utilization',
        'QueueLength',
        'QueuePressure'
    ]

    return df[final_cols]

In [6]:
# Build metadata and apply transformation pipeline to raw dataset
meta_dict, id_map, reverse_id_map = build_parking_metadata(df)

df_transformed = transform_parking_data(df, id_map)

df_transformed.sample(5)

,SystemCodeNumber,VehicleType,TrafficConditionNearby,TimeStamp,IsSpecialDay,TimeCategory,Occupancy,Utilization,QueueLength,QueuePressure
1083,11,2,1,2016-10-08 10:30:00,0,0,488,0.369138,4,0.004785
2736,2,2,0,2016-10-14 15:30:00,0,2,334,0.710638,2,0.014599
1179,12,2,2,2016-10-08 14:00:00,0,2,1058,0.340960,7,0.003417
7759,13,2,1,2016-11-05 15:00:00,0,2,783,0.407813,3,0.002635
4584,11,2,0,2016-10-24 09:30:00,0,0,438,0.331316,2,0.002260


## 📊 Exploratory Data Analysis (EDA)

> This section visualizes key relationships and distributions in the transformed dataset using interactive plots.


In [7]:
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SystemCodeNumber        18368 non-null  int64         
 1   VehicleType             18368 non-null  int64         
 2   TrafficConditionNearby  18368 non-null  int64         
 3   TimeStamp               18368 non-null  datetime64[ns]
 4   IsSpecialDay            18368 non-null  int64         
 5   TimeCategory            18368 non-null  int32         
 6   Occupancy               18368 non-null  int64         
 7   Utilization             18368 non-null  float64       
 8   QueueLength             18368 non-null  int64         
 9   QueuePressure           18368 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(6)
memory usage: 1.3 MB


In [8]:
# 📍 Plot parking locations interactively using Plotly
import plotly.express as px
import pandas as pd

# Get unique parking locations and their metadata
parking_locations = pd.DataFrame.from_dict(meta_dict, orient='index')

# Remove outlier location (index 3)
parking_locations = parking_locations[parking_locations.index != 3]

# Create interactive scatter plot
fig = px.scatter(
    parking_locations,
    x="Latitude",
    y="Longitude",
    size="Capacity",                         # Marker size = capacity
    color=parking_locations.index,           # Color by encoded SystemCodeNumber
    hover_name=parking_locations['SystemCodeNumber'],
    hover_data={
        'Capacity': True,
        'Longitude': ':.4f',
        'Latitude': ':.4f',
        parking_locations.index.name: False
    },
    title="Parking Locations with Capacity",
    labels={parking_locations.index.name: "System Code Number (Encoded)"}
)

# Set plot bounds for focused view
fig.update_layout(
    xaxis_title="Latitude",
    yaxis_title="Longitude",
    legend_title="System Code Number (Encoded)",
    xaxis=dict(range=[26.13, 26.16]),
    yaxis=dict(range=[91.72, 91.75])
)

fig.show()


In [9]:
import plotly.express as px

# Map numeric traffic codes to readable labels
df_transformed['TrafficLabel'] = df_transformed['TrafficConditionNearby'].map({
    0: 'Low',
    1: 'Average',
    2: 'High'
})

# Plot average QueueLength per SystemCodeNumber grouped by traffic level
fig = px.bar(
    df_transformed,
    x='SystemCodeNumber',
    y='QueueLength',
    color='TrafficLabel',
    barmode='group',
    title='Avg QueueLength per Parking Slot by Traffic Condition',
    labels={
        'QueueLength': 'Average QueueLength',
        'SystemCodeNumber': 'Parking Slot',
        'TrafficLabel': 'Traffic Level'
    },
    category_orders={'TrafficLabel': ['Low', 'Average', 'High']},
)

fig.update_layout(xaxis_type='category')
fig.show()

# Drop temporary column to clean up
df_transformed.drop(['TrafficLabel'], axis=1, inplace=True)


In [10]:
import plotly.express as px

# Map time categories to readable labels
df_transformed['TimeLabel'] = df_transformed['TimeCategory'].map({
    0: 'Morning',
    1: 'Noon',
    2: 'Afternoon'
})

# Plot average QueueLength per SystemCodeNumber grouped by Time of Day
fig = px.bar(
    df_transformed,
    x='SystemCodeNumber',
    y='QueueLength',
    color='TimeLabel',
    barmode='group',
    title='Avg QueueLength per Parking Slot by Time of Day',
    labels={
        'QueueLength': 'Average QueueLength',
        'SystemCodeNumber': 'Parking Slot',
        'TimeLabel': 'Time of Day'
    },
    category_orders={'TimeLabel': ['Morning', 'Noon', 'Afternoon']},
)

fig.update_layout(xaxis_type='category')
fig.show()

# Clean up temp column
df_transformed.drop(['TimeLabel'], axis=1, inplace=True)


In [11]:
import plotly.express as px

# Map IsSpecialDay to readable labels
df_transformed['SpecialDayLabel'] = df_transformed['IsSpecialDay'].map({
    0: 'Normal Day',
    1: 'Special Day'
})

# Plot average QueueLength per SystemCodeNumber grouped by Day Type
fig = px.bar(
    df_transformed,
    x='SystemCodeNumber',
    y='QueueLength',
    color='SpecialDayLabel',
    barmode='group',
    title='Avg QueueLength per Parking Slot by Special Day',
    labels={
        'QueueLength': 'Average QueueLength',
        'SystemCodeNumber': 'Parking Slot',
        'SpecialDayLabel': 'Day Type'
    },
    category_orders={'SpecialDayLabel': ['Normal Day', 'Special Day']},
)

fig.update_layout(xaxis_type='category')
fig.show()

# Clean up temporary column
df_transformed.drop(['SpecialDayLabel'], axis=1, inplace=True)


## 🧠 Predictive Modeling

> In this section, we develop and compare multiple models to predict the dynamic parking price based on various engineered features.

### 🔹 Model 1: Baseline Linear Pricing

> A simple linear model that adjusts the base parking price based on occupancy levels. Higher occupancy results in a higher price. This model assumes pricing proportional to demand using a fixed slope `alpha`.


In [12]:
# 🔹 Model 1: Baseline Linear Pricing
def baseline_linear_price(df, meta_dict, base_price=10, alpha=5):
    """
    Calculates parking price based on a simple linear model:
    price = base_price + alpha * (occupancy / capacity)

    Parameters:
    - df: Transformed input DataFrame
    - meta_dict: Metadata dictionary containing 'Capacity' info
    - base_price: Minimum base price
    - alpha: Scaling factor for demand

    Returns:
    - DataFrame with new 'price' column
    """
    df = df.copy()

    # Map capacity from metadata based on SystemCodeNumber
    df['Capacity'] = df['SystemCodeNumber'].map(lambda x: meta_dict[x]['Capacity'])

    # Compute linear price
    df['price'] = base_price + alpha * (df['Occupancy'] / df['Capacity'])

    # Round price for simplicity
    df['price'] = df['price'].round(2)

    # Drop temporary capacity column
    return df.drop(columns=['Capacity'])


In [13]:
# 📊 Apply Model 1: Baseline Linear Pricing
df_base = baseline_linear_price(df_transformed, meta_dict)

In [14]:
# 📈 Plot: Average Parking Price vs Time of Day for Each Parking Location (Model 1)

import plotly.express as px

# Format timestamps to display hour-minute as string
df_base['TimeOfDayStr'] = df_base['TimeStamp'].dt.strftime('%H:%M')

# Order times chronologically
time_order = sorted(df_base['TimeOfDayStr'].unique(), key=lambda x: pd.to_datetime(x, format='%H:%M'))
df_base['TimeOfDayStr'] = pd.Categorical(df_base['TimeOfDayStr'], categories=time_order, ordered=True)

# Group by time and parking slot, compute average price
grouped = (
    df_base
    .groupby(['TimeOfDayStr', 'SystemCodeNumber'], observed=False)['price']
    .mean()
    .reset_index()
)

# Create interactive line plot
fig = px.line(
    grouped,
    x="TimeOfDayStr",
    y="price",
    color="SystemCodeNumber",
    markers=True,
    title="🅿️ Average Parking Price vs Time of Day for Each Parking Location",
    labels={
        "TimeOfDayStr": "Time of Day",
        "price": "Price ($)",
        "SystemCodeNumber": "Parking Spot"
    },
)

fig.update_layout(
    xaxis=dict(tickangle=45),
    width=1300,
    height=600,
    legend_title="SystemCodeNumber",
    hovermode="x unified"
)

fig.show()

### 🧠 Model 2 – Demand-Based Dynamic Pricing

> This model introduces a more dynamic pricing mechanism by combining multiple demand-related factors such as utilization, queue length, traffic conditions, whether it’s a special day, and vehicle type.  
> The final price is adjusted proportionally based on a normalized demand score.


In [15]:
# Model 2: Demand-Based Pricing Function
def demand_based_price(df, base_price=10, α=3, β=0.3, γ=0.5, δ=1, ε=0.5, λ=0.5):
    df = df.copy()

    # 🔸 Step 1: Compute raw demand using a linear combination of influencing factors
    df['raw_demand'] = (
        α * df['Utilization']
        + β * df['QueueLength']
        - γ * df['TrafficConditionNearby']
        + δ * df['IsSpecialDay']
        + ε * df['VehicleType']
    )

    # 🔸 Step 2: Normalize demand between 0 and 1
    min_d, max_d = df['raw_demand'].min(), df['raw_demand'].max()
    df['norm_demand'] = (df['raw_demand'] - min_d) / (max_d - min_d + 1e-6)

    # 🔸 Step 3: Adjust pricing based on normalized demand
    df['price'] = base_price * (1 + λ * df['norm_demand'])

    # 🔸 Step 4: Clip extreme prices to a reasonable range and round
    df['price'] = df['price'].clip(lower=0.5 * base_price, upper=2 * base_price).round(2)

    return df.drop(columns=['raw_demand', 'norm_demand'])


In [16]:
# ✅ Apply demand-based dynamic pricing model
# This model calculates price based on utilization, queue pressure,
# vehicle type, traffic level, time of day, and special day flag.
df_demand = demand_based_price(df_transformed)


In [17]:
# 📈 Plot: Average Parking Price vs Time of Day for Each Parking Location

import plotly.express as px

# Convert TimeStamp to hour-minute string for plotting
df_demand['TimeOfDayStr'] = df_demand['TimeStamp'].dt.strftime('%H:%M')

# Ensure correct time ordering on x-axis
time_order = sorted(df_demand['TimeOfDayStr'].unique(), key=lambda x: pd.to_datetime(x, format='%H:%M'))
df_demand['TimeOfDayStr'] = pd.Categorical(df_demand['TimeOfDayStr'], categories=time_order, ordered=True)

# Group by Time of Day and SystemCodeNumber to calculate mean price
grouped = (
    df_demand
    .groupby(['TimeOfDayStr', 'SystemCodeNumber'])['price']
    .mean()
    .reset_index()
)

# Create interactive line plot
fig = px.line(
    grouped,
    x="TimeOfDayStr",
    y="price",
    color="SystemCodeNumber",
    markers=True,
    title="🅿️ Average Parking Price vs Time of Day for Each Parking Location",
    labels={
        "TimeOfDayStr": "Time of Day",
        "price": "Price ($)",
        "SystemCodeNumber": "Parking Spot"
    },
)

# Enhance layout
fig.update_layout(
    xaxis=dict(tickangle=45),
    width=1300,
    height=600,
    legend_title="SystemCodeNumber",
    hovermode="x unified"
)

fig.show()


/tmp/ipython-input-17-2388128140.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### 🧠 Model 3 – Competitive Price Model (with Location-Aware Adjustments)

> This model incorporates spatial competition by factoring in nearby parking slots. It uses a regression-based price model and includes distance-based adjustments for competitor prices.


In [18]:
# 📌 Utility: Normalize a column using Min-Max scaling
def normalize_minmax_column(col):
    min_val = np.min(col)
    max_val = np.max(col)
    denom = max_val - min_val if max_val != min_val else 1
    return (col - min_val) / denom

# 📌 Utility: Train a simple linear regression model using Normal Equation
def train_regression(X, y):
    # Add bias term (intercept)
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]
    # Compute weights using closed-form solution
    weights = np.linalg.inv(X_bias.T @ X_bias) @ X_bias.T @ y
    return weights

# 📌 Utility: Compute Haversine distance between two lat/lon pairs (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

In [19]:
# 🧠 Add price using a rule-enhanced regression model (competitive pricing)
def add_price(df, meta_dict, base_price=10, peak_multiplier=2.0, min_price=5, max_price=20):
    df = df.copy()

    # Step 1: Normalize queue length
    df['QueueLengthNorm'] = normalize_minmax_column(df['QueueLength'].values)

    # Step 2: Rule-based price estimation
    def handcrafted_price(row):
        # Override for peak demand
        if row['QueueLength'] >= 10 and row['QueuePressure'] >= 1.0:
            return base_price * peak_multiplier

        # Feature-based multiplier (scaled where needed)
        utilization = row['Utilization']
        queue_pressure = row['QueuePressure']
        queue_len_norm = row['QueueLengthNorm']
        vehicle_factor = row['VehicleType'] / 3
        traffic_factor = row['TrafficConditionNearby'] / 2
        time_factor = row['TimeCategory'] / 2
        is_special = row['IsSpecialDay']

        # Feature weights
        w_util = 0.35
        w_queue_p = 0.25
        w_queue_len = 0.1
        w_vehicle = 0.1
        w_traffic = 0.05
        w_time = 0.1
        w_special = 0.05

        # Multiplier based on weighted sum
        multiplier = 1 + (
            w_util * utilization +
            w_queue_p * queue_pressure +
            w_queue_len * queue_len_norm +
            w_vehicle * vehicle_factor +
            w_traffic * traffic_factor +
            w_time * time_factor +
            w_special * is_special
        )

        return base_price * multiplier

    df['price_temp'] = df.apply(handcrafted_price, axis=1)

    # Step 3: Prepare data for linear regression
    features = [
        'Utilization',
        'QueuePressure',
        'QueueLengthNorm',
        'VehicleType',
        'TrafficConditionNearby',
        'IsSpecialDay',
        'TimeCategory',
    ]
    X = df[features].values
    y = df['price_temp'].values

    # Step 4: Fit regression model
    weights = train_regression(X, y)
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]
    preds = X_bias @ weights

    # Clip final prices to allowed range
    df['price'] = np.clip(preds, min_price, max_price).round(2).astype(np.float64)

    # Step 5: Cleanup temp columns
    df.drop(columns=['price_temp', 'QueueLengthNorm'], inplace=True)

    return df, weights

In [20]:
# 🧭 Suggest rerouting to a nearby parking slot if congestion or price conditions are met
def suggest_reroute(
    row,
    df_lookup,
    meta_dict,
    queue_pressure_thresh_high=1.0,
    queue_pressure_thresh_moderate=0.8,
    price_diff_thresh=2.0,
    radius_km=2.5
):
    scn = row['SystemCodeNumber']
    ts = row['TimeStamp']
    qpress = row['QueuePressure']
    curr_price = row['price']

    lat1, lon1 = meta_dict[scn]['Latitude'], meta_dict[scn]['Longitude']
    potential_reroutes = []

    # Compare with other parking slots
    for other_scn, meta in meta_dict.items():
        if other_scn == scn:
            continue

        lat2, lon2 = meta['Latitude'], meta['Longitude']
        dist = haversine(lat1, lon1, lat2, lon2)

        if dist <= radius_km:
            other_row = df_lookup.get((other_scn, ts))
            if other_row:
                other_price = other_row['price']
                other_qpress = other_row['QueuePressure']

                # Condition 1: High congestion at current slot
                if qpress > queue_pressure_thresh_high and other_qpress < qpress:
                    potential_reroutes.append({
                        'scn': other_scn,
                        'dist': dist,
                        'price': other_price,
                        'qpress': other_qpress
                    })

                # Condition 2: Moderate congestion and better price
                elif (
                    queue_pressure_thresh_moderate <= qpress <= queue_pressure_thresh_high and
                    (curr_price - other_price) >= price_diff_thresh and
                    other_qpress < qpress
                ):
                    potential_reroutes.append({
                        'scn': other_scn,
                        'dist': dist,
                        'price': other_price,
                        'qpress': other_qpress
                    })

    # Choose the best reroute candidate
    if potential_reroutes:
        potential_reroutes.sort(key=lambda x: (x['qpress'], x['price'], x['dist']))
        return potential_reroutes[0]['scn']  # Return best alternative

    return None


In [21]:
# ⚙️ Run Competitive Pricing Model
df_comp, weights = add_price(df_transformed, meta_dict)

# 📊 Aggregate to get average price and pressure per (SystemCodeNumber, TimeStamp)
df_agg = df_comp.groupby(['SystemCodeNumber', 'TimeStamp']).agg(
    price=('price', 'mean'),
    QueuePressure=('QueuePressure', 'mean')
).reset_index()

# 🧭 Prepare lookup dictionary for rerouting decisions
df_lookup = df_agg.set_index(['SystemCodeNumber', 'TimeStamp']).to_dict('index')

# 🚦 Apply rerouting logic using suggest_reroute()
df_comp['RerouteTo'] = df_comp.apply(
    lambda row: suggest_reroute(
        row,
        df_lookup,
        meta_dict,
        queue_pressure_thresh_high=1.0,  # Congestion threshold
        price_diff_thresh=2.0            # Price advantage threshold
    ),
    axis=1
)


In [22]:
import plotly.express as px

# 🕒 Format time for x-axis labels (HH:MM)
df_comp['TimeOfDayStr'] = df_comp['TimeStamp'].dt.strftime('%H:%M')

# 📊 Order time values chronologically
time_order = sorted(df_comp['TimeOfDayStr'].unique(), key=lambda x: pd.to_datetime(x, format='%H:%M'))
df_comp['TimeOfDayStr'] = pd.Categorical(df_comp['TimeOfDayStr'], categories=time_order, ordered=True)

# 🧮 Compute average price per TimeOfDay & Parking Spot
grouped = (
    df_comp
    .groupby(['TimeOfDayStr', 'SystemCodeNumber'], observed=False)['price']
    .mean()
    .reset_index()
)

# 📈 Line plot of average price over time for each parking location
fig = px.line(
    grouped,
    x="TimeOfDayStr",
    y="price",
    color="SystemCodeNumber",
    markers=True,
    title="🅿️ Average Parking Price vs Time of Day for Each Parking Location",
    labels={
        "TimeOfDayStr": "Time of Day",
        "price": "Price ($)",
        "SystemCodeNumber": "Parking Spot"
    },
)

# 🧼 Refine layout
fig.update_layout(
    xaxis=dict(tickangle=45),
    width=1300,
    height=600,
    legend_title="SystemCodeNumber",
    hovermode="x unified"
)

# 🚀 Show plot
fig.show()


In [23]:
# ✅ Add a boolean column indicating if rerouting happened
df_comp['Rerouted'] = df_comp['RerouteTo'].notnull()

# 📊 Calculate reroute percentage per parking location
reroute_summary = (
    df_comp
    .groupby('SystemCodeNumber')['Rerouted']
    .mean()
    .sort_values(ascending=False)
)

# 🖨️ Display rerouting percentage per lot (in %)
print(reroute_summary * 100)

SystemCodeNumber
1     19.740854
2      1.905488
6      0.685976
0      0.076220
4      0.000000
3      0.000000
5      0.000000
7      0.000000
8      0.000000
9      0.000000
10     0.000000
11     0.000000
12     0.000000
13     0.000000
Name: Rerouted, dtype: float64


## 🔄 Live Data Streaming with Pathway

> In this section, we implement a real-time streaming pipeline using **Pathway** to compute and aggregate daily parking prices dynamically.

### 📤 Stream Output to CSV and Trigger Execution

> We write the processed price stream to a CSV file and run the pipeline to simulate continuous data flow.

In [24]:
%%capture --no-display
!pip install pathway bokeh --quiet

In [25]:
import pathway as pw                      # For real-time streaming data pipelines
import bokeh.plotting                     # For interactive visualizations
import panel as pn                        # For dashboard and UI elements
from datetime import datetime, timedelta  # For timestamp operations

In [26]:
# Export relevant columns to CSV for use as simulated streaming input
df_transformed[
    [
        "TimeStamp", "SystemCodeNumber", "VehicleType", "TrafficConditionNearby",
        "IsSpecialDay", "TimeCategory", "Occupancy", "Utilization",
        "QueueLength", "QueuePressure"
    ]
].to_csv("parking_stream.csv", index=False)


In [27]:
class ParkingSchema(pw.Schema):
    TimeStamp: str                   # ISO timestamp of the observation
    SystemCodeNumber: int            # Unique parking system identifier
    VehicleType: int                 # Encoded vehicle type (e.g., 0: cycle, 1: bike, etc.)
    TrafficConditionNearby: int     # Encoded traffic condition (0: low, 1: avg, 2: high)
    IsSpecialDay: int               # 1 if special day or holiday, else 0
    TimeCategory: int               # Time bucket (0: Morning, 1: Noon, 2: Afternoon)
    Occupancy: int                  # Number of currently occupied parking spots
    Utilization: float              # Fraction of occupied capacity (Occupancy / Capacity)
    QueueLength: int                # Number of vehicles in the queue
    QueuePressure: float            # Normalized queue congestion level

In [28]:
# Load streaming data from CSV with the defined schema
# Simulates a stream at the rate of 100 rows per second
data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=100
)

In [29]:
@pw.udf
def safe_parse_time(ts: str) -> datetime | None:
    """Safely parses timestamp string to datetime object."""
    try:
        return datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
    except Exception:
        return None

In [30]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.TimeStamp.dt.strptime(fmt),  # full datetime object
    day = data.TimeStamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")  # normalized to day start
).filter(pw.this.t.is_not_none())

In [31]:
# Handcrafted Price Logic
@pw.udf
def compute_temp_price(utilization: float, queue_pressure: float, vehicle_type: int,
                       traffic: int, time_cat: int, base_price: float = 10.0) -> float:
    # Normalize categorical inputs
    vehicle_factor = vehicle_type / 3
    traffic_factor = traffic / 2
    time_factor = time_cat / 2

    # Feature weights
    w_util = 0.4
    w_vehicle = 0.2
    w_traffic = 0.15
    w_time = 0.15
    w_queue = 0.1

    # Compute price multiplier
    multiplier = 1 + (
        w_util * utilization +
        w_vehicle * vehicle_factor +
        w_traffic * traffic_factor +
        w_time * time_factor +
        w_queue * queue_pressure
    )

    return round(base_price * multiplier, 2)


In [32]:
# Regression Pricing using Pre-trained Weights
@pw.udf
def apply_regression(util, queue_press, veh, traffic, special, time_cat) -> float:
    bias = weights[0]
    return round(
        bias +
        weights[1] * util +
        weights[2] * queue_press +
        weights[3] * veh +
        weights[4] * traffic +
        weights[5] * special +
        weights[6] * time_cat,
        2
    )

In [33]:
# Add Price Columns
data_with_price = data_with_time.with_columns(
    price_temp = compute_temp_price(
        pw.this.Utilization,
        pw.this.QueuePressure,
        pw.this.VehicleType,
        pw.this.TrafficConditionNearby,
        pw.this.TimeCategory
    ),
    price = apply_regression(
        pw.this.Utilization,
        pw.this.QueuePressure,
        pw.this.VehicleType,
        pw.this.TrafficConditionNearby,
        pw.this.IsSpecialDay,
        pw.this.TimeCategory
    )
)


In [34]:
# Windowed Aggregation (1-Day Tumbling)
delta_window = (
    data_with_price.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.SystemCodeNumber),
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        day=pw.this._pw_instance[0],
        SystemCodeNumber=pw.this._pw_instance[1],
        price_sum=pw.reducers.sum(pw.this.price),
        price_count=pw.reducers.count()
    )
    .with_columns(
        price=pw.this.price_sum / pw.this.price_count
    )
)


In [35]:
# Output Results to CSV File
pw.io.csv.write(
    delta_window.with_columns(system_id=pw.this.SystemCodeNumber).select(
        pw.this.t, pw.this.price, pw.this.system_id
    ),
    "output_price_stream.csv"
)

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning:

PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations



In [36]:
# Trigger the Pathway Execution
pw.run()

Output()

### 📈 Real-Time Price Visualization using Bokeh

> This section uses **Bokeh** to visualize the live parking price data streamed and aggregated via Pathway.

In [37]:
# Essential imports
import pandas as pd
import panel as pn
import time
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20_14

# ⚙️ Enable Panel extension
pn.extension('bokeh', comm='default', sizing_mode="stretch_width")

In [38]:
# ⏳ Optional wait to allow output file to be written (if needed)
time.sleep(1)

# 📥 Load data
ops = pd.read_csv("output_price_stream.csv")

# 🕒 Parse timestamp, drop bad system_id rows
ops['t'] = pd.to_datetime(ops['t'], format='ISO8601', errors='coerce')
ops = ops[ops['system_id'].between(0, 13)]
ops['system_id'] = ops['system_id'].astype(int)

In [39]:
# 🎨 Prepare system IDs and colors
unique_ids = sorted(ops['system_id'].unique())
colors = Category20_14

# 🧩 Multi-choice selector for system_id
system_selector = pn.widgets.MultiChoice(
    name="Select system_id(s)",
    options=unique_ids,
    value=[unique_ids[0]],
    width=400,
    margin=(0, 0, 10, 0)
)

# 📊 Create plot
plot = figure(
    title="Dynamic Parking Price",
    x_axis_type='datetime',
    width=1200,
    height=600
)

# Initialize data sources and renderers
sources = {}
renderers = {}

for i, sid in enumerate(unique_ids):
    color = colors[i % len(colors)]
    sources[sid] = ColumnDataSource(data=dict(t=[], price=[]))
    renderers[sid] = plot.line(
        't', 'price',
        source=sources[sid],
        color=color,
        legend_label=f"System {sid}"
    )
    renderers[sid].visible = False  # hidden by default

plot.legend.click_policy = "hide"

In [40]:
# 🔁 Define callback to update plot
def update_plot(event=None):
    selected = set(system_selector.value)
    for sid in unique_ids:
        if sid in selected:
            sub_df = ops[ops['system_id'] == sid]
            sources[sid].data = {'t': sub_df['t'], 'price': sub_df['price']}
            renderers[sid].visible = True
        else:
            renderers[sid].visible = False

# 🧠 Register the callback
system_selector.param.watch(update_plot, 'value')
update_plot()  # Initial plot display

In [41]:
# 🧩 Panel layout and display
layout = pn.Column(
    "## 🅿️ Live Parking Price Plot by SystemCodeNumber",
    system_selector,
    pn.pane.Bokeh(plot)
)

layout.servable()

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] MultiChoice(margin=(0, 0, 10, 0), name='Select system_id(s)', options=[np.int64(0), ...], value=[np.int64(0)], width=400)
    [2] Bokeh(figure, sizing_mode='stretch_width')